In [1]:
import pandas as pd
import numpy as np
import datasets 
import transformers
import torch
import torchvision
import torch.utils
import evaluate

In [2]:
imdb = datasets.load_dataset("guebin/imdb-tiny")
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") 
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function,batched=True)
trainer_input = tokenized_imdb['train']
trainer_input

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})

In [3]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)
data_collator


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensor

In [4]:
from transformers import TrainingArguments, Trainer

In [5]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)
image_processor = transformers.VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)
def collate_fn(examples): # examples 는 [Dict, Dict, ...]
    tsr_ntchw = torch.stack([example['video'].permute(1,0,2,3) for example in examples])
    tsrlb_n = torch.tensor([example['label'] for example in examples])
    return dict(pixel_values=tsr_ntchw,labels=tsrlb_n)
metric = evaluate.load("accuracy")

In [6]:
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=imdb["train"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

In [7]:
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[0.03551867, 0.03810254],
       [0.03616748, 0.04375375],
       [0.06581712, 0.04590465],
       [0.04936192, 0.05048798],
       [0.05114323, 0.07685257],
       [0.04310443, 0.01375764],
       [0.03829926, 0.05970999],
       [0.03520145, 0.01527779],
       [0.00428325, 0.09462355],
       [0.049203  , 0.06644786]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), metrics={'test_loss': 0.6981292963027954, 'test_accuracy': 0.3, 'test_runtime': 0.5348, 'test_samples_per_second': 18.698, 'test_steps_per_second': 1.87})

In [8]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
) # 이 과정에서 model이 CUDA로 이동
_batched_data = batch_maker.get_test_dataloader(trainer_input) # trainer_input이 CUDA로 이동
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model.to("cpu") # 경우에 따라 생략 가능
model(**data_collator(single_batch))


SequenceClassifierOutput(loss=tensor(0.7206, grad_fn=<NllLossBackward0>), logits=tensor([[0.0043, 0.0946],
        [0.0492, 0.0664]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## `imdb 데이터`는 문제없이 돌아간다.

In [33]:
food = datasets.load_dataset("guebin/food101-tiny")
image_processor = transformers.AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
normalize = torchvision.transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size), 
    torchvision.transforms.ToTensor(), 
    normalize
])
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples
trainer_input = food['train'].with_transform(transforms)
trainer_input

labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
model = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
imdb["train"] ,food['train']

(Dataset({
     features: ['text', 'label'],
     num_rows: 10
 }),
 Dataset({
     features: ['image', 'label'],
     num_rows: 10
 }))

In [39]:
trainer_input

Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

# 문제발견

In [35]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
) # 이 과정에서 model이 CUDA로 이동
_batched_data = batch_maker.get_test_dataloader(trainer_input) # trainer_input이 CUDA로 이동
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model.to("cpu") # 경우에 따라 생략 가능
model(**data_collator(single_batch))


KeyError: 'image'

In [16]:
trainer.predict(trainer_input)

KeyError: 'image'

```python
examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
```
가 동작하지않음.
`examples['image']` 를 인식하지못한다,


## 이유

In [34]:
trainer.predict??

Signature:
trainer.predict(
    test_dataset: torch.utils.data.dataset.Dataset,
    ignore_keys: Optional[List[str]] = None,
    metric_key_prefix: str = 'test',
) -> transformers.trainer_utils.PredictionOutput
Source:   
    def predict(
        self, test_dataset: Dataset, ignore_keys: Optional[List[str]] = None, metric_key_prefix: str = "test"
    ) -> PredictionOutput:
        """
        Run prediction and returns predictions and potential metrics.

        Depending on the dataset and your use case, your test dataset may contain labels. In that case, this method
        will also return metrics, like in `evaluate()`.

        Args:
            test_dataset (`Dataset`):
                Dataset to run the predictions on. If it is an `datasets.Dataset`, columns not accepted by the
                `model.forward()` method are automatically removed. Has to implement the method `__len__`
            ignore_keys (`List[str]`, *optional*):
                A list of keys in the output of 


`-` Trainer의 내부 데이터 처리 로직 
 : 델에 필요한 입력 데이터만을 유지하고, 나머지 불필요한 컬럼은 제거한다.


In [35]:
import inspect
from transformers import Trainer

get_test_dataloader_method = Trainer.get_test_dataloader

# 메소드의 소스 코드 
print(inspect.getsource(get_test_dataloader_method))


    def get_test_dataloader(self, test_dataset: Dataset) -> DataLoader:
        """
        Returns the test [`~torch.utils.data.DataLoader`].

        Subclass and override this method if you want to inject some custom behavior.

        Args:
            test_dataset (`torch.utils.data.Dataset`, *optional*):
                The test dataset to use. If it is a [`~datasets.Dataset`], columns not accepted by the
                `model.forward()` method are automatically removed. It must implement `__len__`.
        """
        data_collator = self.data_collator

        if is_datasets_available() and isinstance(test_dataset, datasets.Dataset):
            test_dataset = self._remove_unused_columns(test_dataset, description="test")
        else:
            data_collator = self._get_collator_with_removed_columns(data_collator, description="test")

        dataloader_params = {
            "batch_size": self.args.eval_batch_size,
            "collate_fn": data_collator,
            "num_w


### 불필요한 column 제거
```python
if self.args.remove_unused_columns:
    test_dataset = self._remove_unused_columns(test_dataset, "test")
```

## 실제 _remove_unused_columns 함수

```python
def _remove_unused_columns(self, dataset: "datasets.Dataset", description: Optional[str] = None):
    if not self.args.remove_unused_columns:
        return dataset
    self._set_signature_columns_if_needed()
    signature_columns = self._signature_columns

    ignored_columns = list(set(dataset.column_names) - set(signature_columns))
    if len(ignored_columns) > 0:
        dset_description = "" if description is None else f"in the {description} set"
        logger.info(
            f"The following columns {dset_description} don't have a corresponding argument in "
            f"`{self.model.__class__.__name__}.forward` and have been ignored: {', '.join(ignored_columns)}."
            f" If {', '.join(ignored_columns)} are not expected by `{self.model.__class__.__name__}.forward`, "
            " you can safely ignore this message."
        )
```

### 모델의 forward 메소드가 필요로 하지 않는 컬럼을 제거합니다.

##  image는 왜 없어졌나? 필요하지않나?

`-` 실제로 필요한 데이터이지만 Trainer 의 자동 컬럼 제거에 의해 transforms 가 진행된 뒤 pixel_values 가 남고 image 는 삭제됨.

```python

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

trainer_input = food['train'].with_transform(transforms)

```

In [37]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

trainer_input = food['train'].with_transform(transforms)


In [38]:
trainer_input

Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

In [42]:
food = datasets.load_dataset("guebin/food101-tiny")
normalize = torchvision.transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
tokenized_food = food.map(transforms, batched=True)

In [43]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    #del examples["image"]
    return examples
trainer_input = food['train'].with_transform(transforms)
trainer_input

labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
model = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
trainer_input = tokenized_food['train']
trainer_input

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 10
})

In [ ]:
from transformers import default_data_collator

# 데이터 콜레이터 설정
data_collator = default_data_collator


In [45]:

# 데이터셋 변환
tokenized_food = food.map(transforms, batched=True)
trainer_input = tokenized_food['train']

# Trainer 초기화
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=trainer_input,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[ 0.01083915, -0.1678273 , -0.07702327, ...,  0.07300004,
        -0.01142586,  0.03779788],
       [ 0.06300883, -0.11327447, -0.03985846, ...,  0.07129773,
        -0.03080731, -0.09368778],
       [-0.00940686, -0.09504724, -0.03323084, ..., -0.02879822,
         0.07947851,  0.08966758],
       ...,
       [ 0.03146166,  0.00128081, -0.06856059, ..., -0.05244326,
         0.06283719, -0.108992  ],
       [ 0.04974507, -0.05761504, -0.02665417, ...,  0.02063747,
         0.04779366,  0.00535474],
       [ 0.12186253, -0.10484602, -0.04391443, ...,  0.07552588,
        -0.07791385, -0.15710917]], dtype=float32), label_ids=array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), metrics={'test_loss': 4.53817892074585, 'test_accuracy': 0.0, 'test_runtime': 0.9008, 'test_samples_per_second': 11.101, 'test_steps_per_second': 1.11})

# 결론

In [26]:
#기존
trainer_input #은 label ,pixel_values 만 이루어지고 image 는 존재하지않는다.
#현재
trainer_input #은 label ,pixel_values ,image 개가 존재하지않는다.

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 10
})

In [27]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
) # 이 과정에서 model이 CUDA로 이동
_batched_data = batch_maker.get_test_dataloader(trainer_input) # trainer_input이 CUDA로 이동
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model.to("cpu") # 경우에 따라 생략 가능
model(**data_collator(single_batch))


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label', 'pixel_values']


### 특히 batch_maker의 data_collator를 설정할때 항등함수, 즉 lambda x: x를 설정하는 아이디어가 무엇인가?

In [28]:
batch_maker = transformers.Trainer(
    model=model,
    data_collator=lambda x: x
)


`-` 데이터의 원본 형태를 유지하면서 배치화된 데이터가 변형되지 않아 배치 데이터가 올바르게 로드되고 있는지 확인
`-` 배치 데이터가 모델에 올바르게 전달되는지 확인하기위한 방식으로 모델이 전달된다는 데이터가 있는지 알수있다.
`-` data_collator 를 기준으로 디버깅하기 용이하여서

In [30]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"] 
    return examples

# 데이터셋 변환
tokenized_food = food.map(transforms, batched=True)
trainer_input = tokenized_food['train']

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=trainer_input,
    tokenizer=image_processor,
    compute_metrics=compute_metrics
)

# 예측 수행
out = trainer.predict(trainer_input)
print(out)


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label', 'pixel_values']

In [31]:
from transformers import default_data_collator

data_collator = default_data_collator


In [32]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"] 
    return examples

# 데이터셋 변환
tokenized_food = food.map(transforms, batched=True)
trainer_input = tokenized_food['train']

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=trainer_input,
    tokenizer=image_processor,
    compute_metrics=compute_metrics
)

# 예측 수행
out = trainer.predict(trainer_input)
print(out)


PredictionOutput(predictions=array([[-0.04734493, -0.10161442, -0.0098847 , ..., -0.02101815,
         0.00511698,  0.04641831],
       [ 0.03052979, -0.03972539, -0.18334274, ..., -0.16631567,
         0.02186857,  0.11829114],
       [-0.05981214, -0.13781682,  0.08653287, ..., -0.09224211,
         0.05249383,  0.05686734],
       ...,
       [-0.03101005, -0.06273517,  0.05741514, ...,  0.11152172,
         0.09407387,  0.10655267],
       [ 0.01126928,  0.06474207, -0.03547107, ..., -0.01747045,
        -0.0297004 ,  0.03994456],
       [ 0.02790535,  0.02216615,  0.00386669, ...,  0.11898329,
        -0.00387142,  0.07682618]], dtype=float32), label_ids=array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), metrics={'test_loss': 4.573925971984863, 'test_accuracy': 0.0, 'test_runtime': 1.125, 'test_samples_per_second': 8.889, 'test_steps_per_second': 0.889})


In [79]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
) # 이 과정에서 model이 CUDA로 이동
_batched_data = batch_maker.get_test_dataloader(trainer_input) # trainer_input이 CUDA로 이동
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model.to("cpu") # 경우에 따라 생략 가능
model(**data_collator(single_batch))


ImageClassifierOutput(loss=tensor(4.7258, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0498,  0.0364, -0.1015, -0.0719, -0.1263,  0.0108, -0.1014, -0.0776,
          0.0436, -0.0184, -0.0256, -0.0192, -0.0151, -0.1130, -0.2362, -0.1553,
         -0.1682,  0.0228,  0.0898,  0.0566,  0.0190, -0.2582,  0.1571,  0.0485,
         -0.0382,  0.0880,  0.0117, -0.1380, -0.0074, -0.0156,  0.0851, -0.0393,
         -0.1312,  0.1896,  0.0807, -0.0950,  0.2256,  0.0225,  0.0393,  0.2301,
         -0.0288,  0.2195,  0.0834,  0.1193, -0.1078,  0.0050,  0.0663, -0.2811,
          0.1160,  0.0829, -0.0039,  0.1056,  0.0021, -0.1122, -0.0766, -0.0656,
          0.1015, -0.0523,  0.1534, -0.1808, -0.0561, -0.1642,  0.0329,  0.0671,
          0.1264,  0.0734, -0.1053, -0.1624,  0.0800,  0.1169, -0.1177,  0.0813,
          0.0770,  0.0876,  0.1022, -0.0710,  0.0866, -0.0729, -0.0048, -0.0682,
         -0.0097,  0.1124,  0.0557,  0.1330,  0.0959, -0.0199,  0.0706,  0.0066,
          0.1871,  0.1751,  0.1

# 2

소스코드를 살펴보면 구조상

In [75]:
import inspect
from transformers import Trainer

get_test_dataloader_method = Trainer.get_test_dataloader

# 메소드의 소스 코드 
print(inspect.getsource(get_test_dataloader_method))


    def get_test_dataloader(self, test_dataset: Dataset) -> DataLoader:
        """
        Returns the test [`~torch.utils.data.DataLoader`].

        Subclass and override this method if you want to inject some custom behavior.

        Args:
            test_dataset (`torch.utils.data.Dataset`, *optional*):
                The test dataset to use. If it is a [`~datasets.Dataset`], columns not accepted by the
                `model.forward()` method are automatically removed. It must implement `__len__`.
        """
        data_collator = self.data_collator

        if is_datasets_available() and isinstance(test_dataset, datasets.Dataset):
            test_dataset = self._remove_unused_columns(test_dataset, description="test")
        else:
            data_collator = self._get_collator_with_removed_columns(data_collator, description="test")

        dataloader_params = {
            "batch_size": self.args.eval_batch_size,
            "collate_fn": data_collator,
            "num_w

내부에 
```

if is_datasets_available() and isinstance(test_dataset, datasets.Dataset):
            test_dataset = self._remove_unused_columns(test_dataset, description="test")
```

존재


### 불필요한 column 제거
```python
if self.args.remove_unused_columns:
    test_dataset = self._remove_unused_columns(test_dataset, "test")
```

## 실제 _remove_unused_columns 함수

```python
def _remove_unused_columns(self, dataset: "datasets.Dataset", description: Optional[str] = None):
    if not self.args.remove_unused_columns:
        return dataset
    self._set_signature_columns_if_needed()
    signature_columns = self._signature_columns

    ignored_columns = list(set(dataset.column_names) - set(signature_columns))
    if len(ignored_columns) > 0:
        dset_description = "" if description is None else f"in the {description} set"
        logger.info(
            f"The following columns {dset_description} don't have a corresponding argument in "
            f"`{self.model.__class__.__name__}.forward` and have been ignored: {', '.join(ignored_columns)}."
            f" If {', '.join(ignored_columns)} are not expected by `{self.model.__class__.__name__}.forward`, "
            " you can safely ignore this message."
        )
```

In [ ]:
# 

In [78]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"] 
    return examples

# 데이터셋 변환
tokenized_food = food.map(transforms, batched=True)
trainer_input = tokenized_food['train']

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=trainer_input,
    tokenizer=image_processor,
    compute_metrics=compute_metrics
)

# 예측 수행
out = trainer.predict(trainer_input)
print(out)


PredictionOutput(predictions=array([[ 0.00094063,  0.0535494 , -0.04923236, ..., -0.03558214,
        -0.08833199,  0.01511256],
       [-0.00546638, -0.00473674, -0.05026781, ..., -0.0122215 ,
        -0.08843242,  0.10850345],
       [ 0.07712862, -0.04868463,  0.11188272, ..., -0.08693653,
        -0.00782336,  0.05035832],
       ...,
       [-0.10012524,  0.10911529, -0.03048095, ..., -0.0070004 ,
         0.03550508,  0.17775643],
       [-0.04984428,  0.0363589 , -0.1014709 , ...,  0.02122169,
        -0.13342293,  0.1258704 ],
       [-0.20243968,  0.16772076, -0.0733562 , ..., -0.0222971 ,
        -0.11785782,  0.07938986]], dtype=float32), label_ids=array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), metrics={'test_loss': 4.652604103088379, 'test_accuracy': 0.0, 'test_runtime': 1.3629, 'test_samples_per_second': 7.338, 'test_steps_per_second': 3.669})


In [79]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
) # 이 과정에서 model이 CUDA로 이동
_batched_data = batch_maker.get_test_dataloader(trainer_input) # trainer_input이 CUDA로 이동
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model.to("cpu") # 경우에 따라 생략 가능
model(**data_collator(single_batch))


ImageClassifierOutput(loss=tensor(4.7258, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0498,  0.0364, -0.1015, -0.0719, -0.1263,  0.0108, -0.1014, -0.0776,
          0.0436, -0.0184, -0.0256, -0.0192, -0.0151, -0.1130, -0.2362, -0.1553,
         -0.1682,  0.0228,  0.0898,  0.0566,  0.0190, -0.2582,  0.1571,  0.0485,
         -0.0382,  0.0880,  0.0117, -0.1380, -0.0074, -0.0156,  0.0851, -0.0393,
         -0.1312,  0.1896,  0.0807, -0.0950,  0.2256,  0.0225,  0.0393,  0.2301,
         -0.0288,  0.2195,  0.0834,  0.1193, -0.1078,  0.0050,  0.0663, -0.2811,
          0.1160,  0.0829, -0.0039,  0.1056,  0.0021, -0.1122, -0.0766, -0.0656,
          0.1015, -0.0523,  0.1534, -0.1808, -0.0561, -0.1642,  0.0329,  0.0671,
          0.1264,  0.0734, -0.1053, -0.1624,  0.0800,  0.1169, -0.1177,  0.0813,
          0.0770,  0.0876,  0.1022, -0.0710,  0.0866, -0.0729, -0.0048, -0.0682,
         -0.0097,  0.1124,  0.0557,  0.1330,  0.0959, -0.0199,  0.0706,  0.0066,
          0.1871,  0.1751,  0.1